# Pipeline ETL: Camada Prata para Ouro

## 1. Objetivo

Este notebook é responsável pela construção do Data Warehouse (Camada Gold). O processo consiste em:
1.  **Extrair** os dados denormalizados da tabela `orders` (Camada Silver).
2.  **Transformar** os dados aplicando a modelagem dimensional (Star Schema), separando os atributos em tabelas Dimensão e as métricas em uma tabela Fato.
3.  **Carregar** os dados no schema `dw` do PostgreSQL, gerando chaves substitutas (Surrogate Keys) para garantir a integridade referencial e performance de BI.

In [ ]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, Window
import os
from dotenv import load_dotenv
import psycopg2
import warnings

warnings.filterwarnings('ignore')

spark = SparkSession.builder \
    .appName("ETLSilverToGoldOlist") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.5.0") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .config("spark.ui.showConsoleProgress", "false") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
print("SparkSession iniciada.")

## 2. Preparação do Ambiente (Schema DW)

Antes de iniciar a carga, executamos o script DDL (`gold_ddl.sql`) para garantir que o schema `dw` e as tabelas de destino (`dim_...` e `ft_...`) existam e estejam limpas (reset).

In [2]:
def carregar_env():
    env_path = '../../.env'
    load_dotenv(dotenv_path=env_path)
    return {
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "host": os.getenv("DB_HOST"),
        "port": os.getenv("DB_PORT"),
        "dbname": os.getenv("DB_NAME")
    }

def resetar_dw(env_vars):
    ddl_path = "../../DataLayer/gold/gold_ddl.sql"
    print("Executando DDL...")
    try:
        conn = psycopg2.connect(**env_vars)
        conn.autocommit = True
        cur = conn.cursor()
        with open(ddl_path, 'r') as f:
            cur.execute(f.read())
        print("Schema DW recriado com sucesso.")
    except Exception as e:
        print(f"Erro no DDL: {e}")
        raise
    finally:
        if 'cur' in locals(): cur.close()
        if 'conn' in locals(): conn.close()

env_vars = carregar_env()
resetar_dw(env_vars)

Executando DDL...
Schema DW recriado com sucesso.


## 3. Extração (Leitura da Silver)

Lemos os dados da "Tabelona" (`public.orders`) diretamente do PostgreSQL para um DataFrame Spark. Esta é a nossa fonte única da verdade.

In [3]:
jdbc_url = f"jdbc:postgresql://{env_vars['host']}:{env_vars['port']}/{env_vars['dbname']}"
jdbc_props = {"user": env_vars['user'], "password": env_vars['password'], "driver": "org.postgresql.Driver"}

print("Lendo dados da camada Silver...")
df_silver = spark.read.jdbc(jdbc_url, "public.orders", properties=jdbc_props)
df_silver.cache()
print(f"Linhas carregadas: {df_silver.count()}")

Lendo dados da camada Silver...
Linhas carregadas: 88981


## 4. Processamento das Dimensões (Transform & Load)

Nesta etapa, "fatiamos" os dados da Silver para criar as tabelas de dimensão. Para cada dimensão:
1.  Selecionamos colunas distintas.
2.  Geramos uma Surrogate Key (`sk_`) sequencial.
3.  Carregamos na tabela correspondente no PostgreSQL (`dw.dim_...`).

In [4]:
print("Processando Dimensões (Cliente, Vendedor, Produto, Pagamento)...")

dim_cliente = df_silver.select(
    F.col("customer_unique_id").alias("nk_id_cliente"),
    F.col("customer_city").alias("nm_cidade"),
    F.col("customer_state").alias("sg_estado")
).distinct()
dim_cliente = dim_cliente.withColumn("sk_cliente", F.row_number().over(Window.orderBy("nk_id_cliente")))

dim_vendedor = df_silver.select(
    F.col("seller_id").alias("nk_id_vendedor"),
    F.col("seller_city").alias("nm_cidade"),
    F.col("seller_state").alias("sg_estado")
).distinct()
dim_vendedor = dim_vendedor.withColumn("sk_vendedor", F.row_number().over(Window.orderBy("nk_id_vendedor")))

dim_produto = df_silver.select(
    F.col("product_id").alias("nk_id_produto"),
    F.col("product_category_name").alias("nm_categoria")
).distinct()
dim_produto = dim_produto.withColumn("sk_produto", F.row_number().over(Window.orderBy("nk_id_produto")))

dim_pagamento = df_silver.select(
    F.col("payment_type").alias("ds_tipo_pagamento"),
    F.col("payment_installments").alias("nr_parcelas")
).distinct()
dim_pagamento = dim_pagamento.withColumn("sk_pagamento", F.row_number().over(Window.orderBy("ds_tipo_pagamento", "nr_parcelas")))

print("Carregando Dimensões no PostgreSQL...")
dim_cliente.select("nk_id_cliente", "nm_cidade", "sg_estado").write.jdbc(jdbc_url, "dw.dim_cliente", "append", jdbc_props)
dim_vendedor.select("nk_id_vendedor", "nm_cidade", "sg_estado").write.jdbc(jdbc_url, "dw.dim_vendedor", "append", jdbc_props)
dim_produto.select("nk_id_produto", "nm_categoria").write.jdbc(jdbc_url, "dw.dim_produto", "append", jdbc_props)
dim_pagamento.select("ds_tipo_pagamento", "nr_parcelas").write.jdbc(jdbc_url, "dw.dim_pagamento", "append", jdbc_props)
print("Carga de Dimensões concluída.")

Processando Dimensões (Cliente, Vendedor, Produto, Pagamento)...
Carregando Dimensões no PostgreSQL...
Carga de Dimensões concluída.


### 4.1. Dimensão Tempo

A Dimensão Tempo é gerada a partir das datas de compra existentes. Enriquecemos os dados extraindo atributos como Dia, Mês, Ano, Trimestre, Semestre e Flag de Fim de Semana para facilitar a análise temporal no dashboard.

In [5]:
print("Processando Dimensão Tempo...")

df_datas = df_silver.select(F.to_date("order_purchase_timestamp").alias("data_completa")).distinct()

dim_tempo = df_datas.select(
    F.col("data_completa"),
    F.date_format("data_completa", "yyyyMMdd").cast("int").alias("sk_tempo"),
    F.year("data_completa").alias("nr_ano"),
    F.month("data_completa").alias("nr_mes"),
    F.date_format("data_completa", "MMMM").alias("nm_mes"),
    F.dayofmonth("data_completa").alias("nr_dia"),
    F.quarter("data_completa").alias("nr_trimestre"),
    F.date_format("data_completa", "EEEE").alias("nm_dia_semana"),
    (F.dayofweek("data_completa").isin([1, 7])).alias("fl_fim_de_semana")
)

dim_tempo.write.jdbc(jdbc_url, "dw.dim_tempo", "append", jdbc_props)
print("Dimensão Tempo carregada.")

Processando Dimensão Tempo...
Dimensão Tempo carregada.


## 5. Processamento da Tabela Fato (Transform & Load)

Esta é a etapa final de integração. Realizamos o **JOIN** da tabela Silver original com as novas Dimensões criadas para substituir as chaves naturais (IDs originais) pelas chaves artificiais (`sk_`) do DW.

Selecionamos apenas as chaves estrangeiras e as métricas numéricas para compor a tabela `dw.ft_vendas`.

In [6]:
print("Processando Tabela Fato Vendas...")

df_fato_base = df_silver.withColumn("data_join", F.to_date("order_purchase_timestamp"))

df_fato = df_fato_base \
    .join(dim_cliente, df_fato_base.customer_unique_id == dim_cliente.nk_id_cliente, "left") \
    .join(dim_vendedor, df_fato_base.seller_id == dim_vendedor.nk_id_vendedor, "left") \
    .join(dim_produto, df_fato_base.product_id == dim_produto.nk_id_produto, "left") \
    .join(dim_tempo, df_fato_base.data_join == dim_tempo.data_completa, "left") \
    .join(dim_pagamento, 
          (df_fato_base.payment_type == dim_pagamento.ds_tipo_pagamento) & 
          (df_fato_base.payment_installments == dim_pagamento.nr_parcelas), "left")

ft_vendas = df_fato.select(
    F.col("sk_cliente"),
    F.col("sk_vendedor"),
    F.col("sk_produto"),
    F.col("sk_tempo"),
    F.col("sk_pagamento"),
    F.col("order_id").alias("nk_id_pedido"),
    (F.col("price") + F.col("freight_value")).alias("vlr_total"),
    F.col("freight_value").alias("vlr_frete"),
    F.col("price").alias("vlr_item"),
    F.col("delivery_days").alias("qtd_dias_entrega"),
    F.col("review_score").alias("nota_avaliacao"),
    F.col("is_delivery_late").alias("fl_atraso")
)

print("Carregando Fato Vendas...")
ft_vendas.write.jdbc(jdbc_url, "dw.ft_vendas", "append", jdbc_props)
print("Carga da Camada Gold concluída com sucesso.")

Processando Tabela Fato Vendas...
Carregando Fato Vendas...
Carga da Camada Gold concluída com sucesso.


In [7]:
spark.stop()